In [21]:
# Step 1: Install necessary libraries
!pip install pandas numpy scikit-learn tensorflow

# Import necessary libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
import warnings

# Suppress warnings
warnings.filterwarnings('ignore')

# Load the dataset
data = pd.read_csv('/content/Alphabets_data.csv')  # Ensure the file is uploaded to Colab
print(data.head())
print(data.info())

# Check for missing values
if data.isnull().values.any():
    print("Missing values detected. Filling missing values with mean.")
    data.fillna(data.mean(), inplace=True)  # Fill missing values with mean

# Separate features and labels
features = data.iloc[:, :-1]  # All columns except the last
labels = data.iloc[:, -1]      # Last column

# Handle categorical features if present
for col in features.select_dtypes(include=['object']).columns:
    features[col] = features[col].astype('category').cat.codes  # Convert to categorical codes

# Normalize the features
scaler = MinMaxScaler()
scaled_features = scaler.fit_transform(features)

# Encode the labels
label_encoder = LabelEncoder()
encoded_labels = label_encoder.fit_transform(labels)

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(scaled_features, encoded_labels, test_size=0.2, random_state=42)

print(f"Training set size: {X_train.shape[0]}, Test set size: {X_test.shape[0]}")


  letter  xbox  ybox  width  height  onpix  xbar  ybar  x2bar  y2bar  xybar  \
0      T     2     8      3       5      1     8    13      0      6      6   
1      I     5    12      3       7      2    10     5      5      4     13   
2      D     4    11      6       8      6    10     6      2      6     10   
3      N     7    11      6       6      3     5     9      4      6      4   
4      G     2     1      3       1      1     8     6      6      6      6   

   x2ybar  xy2bar  xedge  xedgey  yedge  yedgex  
0      10       8      0       8      0       8  
1       3       9      2       8      4      10  
2       3       7      3       7      3       9  
3       4      10      6      10      2       8  
4       5       9      1       7      5      10  
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 17 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   letter  20000 non-null  object
 1   xbo

In [22]:
# Import necessary libraries for model building
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# Define the model
model = Sequential()
model.add(Dense(128, activation='relu', input_shape=(X_train.shape[1],)))  # Input layer
model.add(Dense(64, activation='relu'))  # Hidden layer
model.add(Dense(len(np.unique(y_train)), activation='softmax'))  # Output layer for multi-class classification

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_split=0.2)

# Evaluate the model
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f'Test Accuracy: {test_accuracy:.4f}')


Epoch 1/50
400/400 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.3885 - loss: 1.9360 - val_accuracy: 0.4109 - val_loss: 1.5700
Epoch 2/50
400/400 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.4269 - loss: 1.5309 - val_accuracy: 0.4591 - val_loss: 1.4527
Epoch 3/50
400/400 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.4575 - loss: 1.4360 - val_accuracy: 0.4547 - val_loss: 1.3916
Epoch 4/50
400/400 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.4633 - loss: 1.4048 - val_accuracy: 0.4837 - val_loss: 1.3570
Epoch 5/50
400/400 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.4825 - loss: 1.3517 - val_accuracy: 0.4994 - val_loss: 1.3148
Epoch 6/50
400/400 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.4913 - loss: 1.3189 - val_accuracy: 0.5106 - val_loss: 1.2861
Epoch 7/50
400/400 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.5055 - loss: 1.2895 - val_accuracy: 0.5213 - val_loss: 1.2622
Epoch 8/50
400/400 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.5148 - loss: 1.2693 - val_accuracy: 0.

In [27]:
# Install TensorFlow if not already installed
!pip install tensorflow

from sklearn.model_selection import ParameterGrid
from sklearn.metrics import classification_report

# Function to create and compile a model
def create_model(hidden_layers=1, neurons=64, activation='relu'):
    model = Sequential()
    model.add(Dense(neurons, activation=activation, input_shape=(X_train.shape[1],)))
    for _ in range(hidden_layers - 1):
        model.add(Dense(neurons, activation=activation))
    model.add(Dense(len(np.unique(y_train)), activation='softmax'))

    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

# Define the grid of hyperparameters to search
param_grid = {
    'hidden_layers': [1, 2],
    'neurons': [32, 64, 128],
    'activation': ['relu', 'tanh']
}

# Create all combinations of hyperparameters
grid = ParameterGrid(param_grid)

best_score = 0
best_params = {}

# Loop through each combination of parameters
for params in grid:
    print(f"Training model with params: {params}")

    # Create and train the model
    model = create_model(hidden_layers=params['hidden_layers'],
                         neurons=params['neurons'],
                         activation=params['activation'])

    model.fit(X_train, y_train, epochs=50, batch_size=32, verbose=0)

    # Evaluate the model
    test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=0)

    print(f"Test Accuracy: {test_accuracy:.4f}")

    # Check if this model is the best so far
    if test_accuracy > best_score:
        best_score = test_accuracy
        best_params = params

# Print the best parameters and best score
print(f"Best: {best_score:.4f} using {best_params}")


Training model with params: {'activation': 'relu', 'hidden_layers': 1, 'neurons': 32}
Test Accuracy: 0.5073
Training model with params: {'activation': 'relu', 'hidden_layers': 1, 'neurons': 64}
Test Accuracy: 0.5397
Training model with params: {'activation': 'relu', 'hidden_layers': 1, 'neurons': 128}
Test Accuracy: 0.5715
Training model with params: {'activation': 'relu', 'hidden_layers': 2, 'neurons': 32}
Test Accuracy: 0.5748
Training model with params: {'activation': 'relu', 'hidden_layers': 2, 'neurons': 64}
Test Accuracy: 0.6030
Training model with params: {'activation': 'relu', 'hidden_layers': 2, 'neurons': 128}
Test Accuracy: 0.6488
Training model with params: {'activation': 'tanh', 'hidden_layers': 1, 'neurons': 32}
Test Accuracy: 0.4910
Training model with params: {'activation': 'tanh', 'hidden_layers': 1, 'neurons': 64}
Test Accuracy: 0.4875
Training model with params: {'activation': 'tanh', 'hidden_layers': 1, 'neurons': 128}
Test Accuracy: 0.4767
Training model with param

In [29]:
# Make predictions on the test set
y_pred = np.argmax(best_model.predict(X_test), axis=-1)

# Get unique classes from y_test
unique_classes = np.unique(y_test)

# Generate classification report
report = classification_report(y_test, y_pred,
                               labels=unique_classes,
                               target_names=[str(cls) for cls in unique_classes])

print("Classification Report:\n", report)


125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
Classification Report:
               precision    recall  f1-score   support

           0       0.00      0.00      0.00         1
           1       0.44      1.00      0.62         4
           2       0.00      0.00      0.00         6
           3       0.55      0.39      0.45        31
           4       0.56      0.48      0.51        94
           5       0.57      0.54      0.55       196
           6       0.55      0.58      0.56       353
           7       0.58      0.57      0.57       712
           8       0.76      0.83      0.80      1596
           9       0.58      0.51      0.54       485
          10       0.55      0.46      0.50       308
          11       0.59      0.61      0.60       175
          12       0.62      0.17      0.27        29
          13       0.00      0.00      0.00         9
          14       0.33      1.00      0.50         1

    accuracy                           0.65      4000
   macro avg   

In [42]:
# Import necessary libraries
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score
from sklearn.preprocessing import LabelEncoder, MinMaxScaler

# Load your data
df = pd.read_csv('/content/Alphabets_data.csv')  # Ensure this points to your CSV file

# Check the first few rows and column names of the DataFrame
print(df.head())
print(df.columns)  # This will help you find the correct column names

# Replace 'labels' with the actual target column name after checking
target_column_name = 'letter'  # Change this to your actual target column name

# Assuming 'actual_target_column' is the target column and the rest are features
features = df.drop(columns=[target_column_name])
labels = df[target_column_name]


# Encode the labels if they are categorical
le = LabelEncoder()
encoded_labels = le.fit_transform(labels)

# Normalize the features
scaler = MinMaxScaler()
scaled_features = scaler.fit_transform(features)

# Split the dataset into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(scaled_features, encoded_labels, test_size=0.2, random_state=42)

# Define the model
model = RandomForestClassifier()

# Define the parameter grid for hyperparameter tuning
param_grid = {
    'n_estimators': [50, 100],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5],
}

# Initialize GridSearchCV
grid_search = GridSearchCV(model, param_grid, cv=3, scoring='accuracy')

# Fit the model
grid_search.fit(X_train, y_train)

# Get the best parameters and score
best_model = grid_search.best_estimator_
best_score = grid_search.best_score_
print("Best Parameters: ", grid_search.best_params_)
print("Best Cross-Validation Score: ", best_score)

# Evaluate on the validation set
y_pred = best_model.predict(X_val)
val_accuracy = accuracy_score(y_val, y_pred)
print("Validation Accuracy: ", val_accuracy)

# Generate classification report
report = classification_report(y_val, y_pred, target_names=le.classes_)
print("Classification Report:\n", report)


  letter  xbox  ybox  width  height  onpix  xbar  ybar  x2bar  y2bar  xybar  \
0      T     2     8      3       5      1     8    13      0      6      6   
1      I     5    12      3       7      2    10     5      5      4     13   
2      D     4    11      6       8      6    10     6      2      6     10   
3      N     7    11      6       6      3     5     9      4      6      4   
4      G     2     1      3       1      1     8     6      6      6      6   

   x2ybar  xy2bar  xedge  xedgey  yedge  yedgex  
0      10       8      0       8      0       8  
1       3       9      2       8      4      10  
2       3       7      3       7      3       9  
3       4      10      6      10      2       8  
4       5       9      1       7      5      10  
Index(['letter', 'xbox', 'ybox', 'width', 'height', 'onpix', 'xbar', 'ybar',
       'x2bar', 'y2bar', 'xybar', 'x2ybar', 'xy2bar', 'xedge', 'xedgey',
       'yedge', 'yedgex'],
      dtype='object')
Best Parameters:  {'max_de